In [1]:
"""
    MYSQL SQL문 연습....
    create & insert 후, 
    조건에 따른 데이터 조회(SELECT) 연습
    다양한 조건 (GROUPBY, JOIN 등 사용)
"""

'\n    MYSQL SQL문 연습....\n    create & insert 후, \n    조건에 따른 데이터 조회(SELECT) 연습\n    다양한 조건 (GROUPBY, JOIN 등 사용)\n'

In [2]:
import pymysql
import pandas as pd
import numpy as np
from config import *
import uuid
import datetime 
from itertools import product

# 기본 쿼리 함수 정의

In [3]:
def query(sql, data):
    """
    DB에 SQL문 실행(query: 데이터베이스에 정보 요청)
       Args:
           sql(str): sql 구문
           data(list or tuple): sql 파라미터. ex) [val1, val2 ...]
       Returns:
           query 결과. 결과가 없으면 빈 tuple 리턴
    """
    # connect(): DB와 연결
    db = pymysql.Connect(host=MYSQL_HOST, 
                     user=MYSQL_USER, 
                     password=MYSQL_PASSWORD, 
                     database=MYSQL_DATABASE,
                     autocommit=MYSQL_AUTOCOMMIT, # Query 실행 후 자동 commit(실제 insert, update 반영)
                     charset=MYSQL_CHARSET, # 인코딩 정보
                     cursorclass=pymysql.cursors.DictCursor # DB를 조회한 결과를 Column 명이 Key인 Dictionary로 저장
                    )
    cursor = db.cursor()

    cursor.execute(sql, data) 
    result = cursor.fetchall() # fetchall(): 모든 데이터를 한 번에 가져옴
    db.close()
    return result


def many_query(sql, data):
    """
    DB에 INSERTL sql문 다중 실행
       Args:
           sql(str): INSERT sql 구문
           data(list or tuple): 다중 sql 파라미터. ex) [[val11, val12], [val21, val22]]
       Returns:
           query 결과. 결과가 없으면 빈 tuple 리턴
    """
    
    # connect(): DB와 연결
    db = pymysql.Connect(host=MYSQL_HOST, 
                     user=MYSQL_USER, 
                     password=MYSQL_PASSWORD, 
                     database=MYSQL_DATABASE,
                     autocommit=MYSQL_AUTOCOMMIT, # Query 실행 후 자동 commit(실제 insert, update 반영)
                     charset=MYSQL_CHARSET, # 인코딩 정보
                     cursorclass=pymysql.cursors.DictCursor # DB를 조회한 결과를 Column 명이 Key인 Dictionary로 저장
                    )
    cursor = db.cursor()

    cursor.executemany(sql, data) 
    result = cursor.fetchall() # fetchall(): 모든 데이터를 한 번에 가져옴
    db.close()
    return result

# DB에 CREATE 하기
1. index: 자동 increase, Primary Key
2. uuid: uuid 36자리 고정, unique key
3. reg_date: 등록 일자(TIMESTAMP)
4. name: 이름
5. email: 이메일 주소
6. phone: 폰 넘버 13자리 고정
7. label: JSON 형태 text
8. mark: JSON 형태 text()
9. ext01: 여분 필드(추후 추가시 사용)

- CHAR: 길이가 고정 혹은 일정 이하일 경우 설정
- TEXT: 길이가 고정적이지 않을 경우 혹은 길 것 같은 경우

In [4]:
# mysql에 테이블 create 코드
query(
      "CREATE TABLE `" + MYSQL_TABLENAME +
    """` (
    `index` BIGINT(20) NOT NULL AUTO_INCREMENT,
    `uuid` CHAR(36) NOT NULL DEFAULT '' COLLATE 'utf8_general_ci',
    `reg_date` TIMESTAMP NOT NULL DEFAULT current_timestamp() ON UPDATE current_timestamp(),
    `name` CHAR(255) NOT NULL COLLATE 'utf8_general_ci',
    `email` CHAR(255) NOT NULL COLLATE 'utf8_general_ci',
    `phone` CHAR(13) NOT NULL COLLATE 'utf8_general_ci',
    `energy` INT(11) UNSIGNED ZEROFILL NOT NULL,
    `label` MEDIUMTEXT NOT NULL COLLATE 'utf8_general_ci',
    `mark` TEXT(65535) NOT NULL COLLATE 'utf8_general_ci',
    `ext01` TEXT(65535) NOT NULL COLLATE 'utf8_general_ci',
    PRIMARY KEY (`index`) USING BTREE,
    UNIQUE INDEX `uuid` (`uuid`) USING BTREE
)
COLLATE='utf8_general_ci'
ENGINE=InnoDB
;

      """, ())

()

In [5]:
query("SELECT * FROM " + MYSQL_TABLENAME+ ";", ())

()

# MYSQL INSERT 하기
- 가정 1시간 마다 데이터 전송하여 저장
- energy 필드는 랜덤
- label은 추가 데이터 정보
- mark, ext01은 empty

In [6]:
np.random.seed(0) # generate 이전 random seed 설정

In [7]:
def generate_data(hour = 0, name = "홍길동", email="hong@mysqltest.com", phone = "010-1234-5678"):
    result = []
    uuid_hex = uuid.uuid1().hex
    
    now = datetime.datetime.utcnow().timestamp() - hour*60*60 # utc로 현재 시간 얻고 계산
    reg_date = str(datetime.datetime.fromtimestamp(now))
    
    
    name = name
    email = email
    phone = phone
    energy = np.random.randint(100) # 100이해 random int 생성
    
    label_search_list = ['youtube', 'github', 'naver']
    
    # JSON 형태는 ""를 사용해야함
    label = '[{"search": "' + label_search_list[np.random.randint(3)] + '"}, {"viewing time": "' + str(np.random.randint(100)) + '"}]'
    mark = ""
    ext01 = ""
    result = [
        uuid_hex,
        reg_date,
        name,
        email,
        phone,
        energy,
        label,
        mark,
        ext01
    ]
    return result

## generate simulation data

In [8]:
name_list = ['Kim', 'Lee', 'Choi']
email_list = ['Kim@mysql.com', 'Lee@mysql.com', 'Choi@mysql.com']
phone_list = ['010-1234-5678', '010-4567-8910', '010-9876-5432']
data = []
for idx, nl in enumerate(name_list):
    for i in range(48): # 48: 3일 전까지 데이터 생성
        data.append(generate_data(hour=i, name=name_list[idx], email=email_list[idx], phone=phone_list[idx]))

## 생성후 time 순으로 정렬

In [9]:
def sortby(x, idx = 0, direc = 'ASC'):
    if direc == 'ASC':
        return sorted(x, key=lambda x: x[idx], reverse = False)
    elif direc == 'DESC':
        return sorted(x, key=lambda x: x[idx], reverse = True)
data = sortby(data, idx = 1, direc = 'DESC')

In [10]:
# # 1개 insert test
# sql = "INSERT INTO " + MYSQL_TABLENAME + """ (uuid, reg_date, name, email, phone, energy, label, mark, ext01) 
# VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
# data = generate_data()
# print(data)
# query(sql, data)

## data insert using many_query

In [11]:
sql = "INSERT INTO " + MYSQL_TABLENAME + """ (uuid, reg_date, name, email, phone, energy, label, mark, ext01) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
many_query(sql, data)

()

## check data

In [12]:
result = query("SELECT * FROM " + MYSQL_TABLENAME +" where name=%s;", ['Kim'])
result_df = pd.DataFrame(result)

In [13]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   index     48 non-null     int64         
 1   uuid      48 non-null     object        
 2   reg_date  48 non-null     datetime64[ns]
 3   name      48 non-null     object        
 4   email     48 non-null     object        
 5   phone     48 non-null     object        
 6   energy    48 non-null     int64         
 7   label     48 non-null     object        
 8   mark      48 non-null     object        
 9   ext01     48 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 3.9+ KB


In [14]:
result_df.head()

,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
1,6,b1773edab46311ed865640b076a1ec6b,2023-02-24 15:52:59,Kim,Kim@mysql.com,010-1234-5678,67,"[{""search"": ""github""}, {""viewing time"": ""83""}]",,
2,9,b17765ebb46311ed924f40b076a1ec6b,2023-02-24 14:52:59,Kim,Kim@mysql.com,010-1234-5678,21,"[{""search"": ""naver""}, {""viewing time"": ""36""}]",,
3,12,b17765ecb46311ed87ad40b076a1ec6b,2023-02-24 13:52:59,Kim,Kim@mysql.com,010-1234-5678,87,"[{""search"": ""naver""}, {""viewing time"": ""88""}]",,
4,15,b17765edb46311edaaeb40b076a1ec6b,2023-02-24 12:52:59,Kim,Kim@mysql.com,010-1234-5678,88,"[{""search"": ""youtube""}, {""viewing time"": ""58""}]",,
